In [1]:
from dataloader import Dataloader
import pandas as pd
import os

pd.options.mode.chained_assignment = None  # default='warn'


In [2]:
# Load Data
path = os.getcwd() + "/../data/"

# load data which creates baskets and coupons
data = Dataloader(path)

In [3]:
# Create Categories before CV as they don't change and consume time otherwise
data.create_category_table(2000)

In [4]:
# weeks = [87, 88, 89]
weeks = [89]
# shopper_list = list(range(2000))
# shopper_chunks = [shopper_list[i:i + 100] for i in range(0, len(shopper_list), 100)]
shopper_chunks = [list(range(100))]

for week in weeks:
    for shopper in shopper_chunks:
        # train-test-split
        data.train_test_split(week, shopper)

        # data add categories 
        data.add_categories()

        # create features
        data.create_feature_dict()

        # combine everything
        X_train, y_train, X_test, y_test = data.make_featured_data()
        
        

In [5]:
X_train.head()

,shopper,product,price,category,discount,coupon,original_price,total_count_of_product,reordered_product,category_count,...,coupon_in_same_category,avg_categorical_discount,ratio_of_reordered_products,ratio_of_reordered_categories,average_price_per_shopper,average_basket_size,unique_products_per_shopper,unique_categories_per_shopper,weeks_since_prior_product_order,weeks_since_prior_category_order
0,0,0,688.0,19,0.0,No,688,0,0.0,8,...,Yes,171.269413,0.722222,0.960000,587.203947,8.539326,54,25,1,1
1,1,0,688.0,19,0.0,No,688,6,1.0,28,...,Yes,171.269413,0.830986,1.000000,584.065350,7.393258,71,24,1,1
2,2,0,688.0,19,0.0,No,688,0,0.0,53,...,No,171.269413,0.700000,0.954545,571.918919,5.820225,60,22,1,1
3,3,0,688.0,19,0.0,No,688,0,0.0,37,...,No,171.269413,0.714286,1.000000,561.846361,8.337079,91,24,1,1
4,4,0,688.0,19,0.0,No,688,0,0.0,8,...,No,171.269413,0.764706,1.000000,583.750455,6.168539,34,20,1,1


In [6]:
X_train.sample(20)

,shopper,product,price,category,discount,coupon,original_price,total_count_of_product,reordered_product,category_count,...,coupon_in_same_category,avg_categorical_discount,ratio_of_reordered_products,ratio_of_reordered_categories,average_price_per_shopper,average_basket_size,unique_products_per_shopper,unique_categories_per_shopper,weeks_since_prior_product_order,weeks_since_prior_category_order
2052742,42,230,417.0,22,0.0,No,417,0,0.0,19,...,No,113.143505,0.821429,1.000000,584.577279,8.505618,84,25,58,6
1923087,87,216,693.0,1,0.0,No,693,0,0.0,15,...,No,155.005676,0.774648,1.000000,580.573237,6.213483,71,24,7,2
1394251,51,156,575.0,14,0.0,No,575,0,0.0,16,...,No,139.421583,0.728814,0.960000,585.395712,5.764045,59,25,59,5
1683545,45,189,506.0,5,0.0,No,506,5,1.0,28,...,No,141.976775,0.716418,1.000000,579.360502,7.168539,67,25,15,7
563244,44,63,719.0,18,0.0,No,719,0,0.0,70,...,Yes,180.054079,0.736111,0.960000,595.849719,8.000000,72,25,26,2
585320,20,65,719.0,18,0.0,No,719,0,0.0,7,...,No,180.054079,0.745098,1.000000,581.853704,6.067416,51,23,69,18
378890,90,42,527.0,10,0.0,No,527,0,0.0,14,...,No,136.187085,0.710145,0.960000,578.817803,8.078652,69,25,51,1
277678,78,31,784.0,20,0.0,No,784,0,0.0,21,...,Yes,172.068348,0.662338,0.958333,565.044304,7.101124,77,24,18,1
1985221,21,223,729.0,9,0.0,No,729,0,0.0,38,...,No,158.831726,0.846154,0.956522,585.085420,7.629213,65,23,6,1
1129484,84,126,598.0,16,0.0,No,598,0,0.0,21,...,No,153.728374,0.738462,1.000000,553.753228,7.831461,65,24,81,4
